<a href="https://colab.research.google.com/github/MichaelHMorei/desafios/blob/main/DesafioAlexandre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

In [2]:
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [3]:
os.environ["SPARK_HOME"]

'/content/spark-3.2.1-bin-hadoop2.7'

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
pip install spark-nlp==2.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 5.1 MB/s 


In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.2.1


In [7]:
#Salvando/Lendo arquivos no google drive
from google.colab import drive
drive.mount("/content/gdrive")

# Lendo os arquivos e passando para uma variavel
df_covid = spark.read.csv("/content/gdrive/My Drive/CovidDataset/caso_full_covid.csv",inferSchema = 'True',header ='True') 


Mounted at /content/gdrive


In [ ]:
# Unzip fonte de dados eventos_de_fluxo.csv
#with gzip.open("/content/gdrive/My Drive/DesafioAlexandre/eventos_de_fluxo.csv.gz", 'rb') as entrada:
     # with gzip.open("/content/gdrive/My Drive/DesafioAlexandre/eventos_de_fluxo.csv", 'wb') as saida:
     #  shutil.copyfileobj(entrada,saida)

NameError: ignored

In [8]:
#Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark	
from pyspark.sql.types import *
import pandas as pd
import requests as r
import json
from datetime import *
from pyspark.sql import functions as Func
import pytz 

In [ ]:
# Obtendo a data/hora atual
 
date1 = datetime.now()
date_atual = date1.strftime('%d/%m/%Y')
hora_Atual = date1.strftime('%H:%M:%S')

print('#### Processamento iniciado dia: '+date_atual+' as: '+hora_Atual+' ####' )

#### Processamento iniciado dia: 08/06/2022 as: 22:35:59 ####


In [10]:
# Transformando os arquivos em Dataframes.

#Leando a fonte de dados eventos_de_fluxo.csv
#Modificando os DataType 
eventosSchema = StructType([StructField("codigo",StringType(),True),
                          StructField("datetime",StringType(),False),
                          StructField("codigo_concorrente",LongType(),False)])

eventos = spark.read.csv("/content/gdrive/My Drive/DesafioAlexandre/eventos_de_fluxo.csv.gz", sep=',' , inferSchema = 'True', schema = eventosSchema, header ='True')

#Leando a fonte de dados concorrentes.cs
concorrentesSchema = StructType(
                          [StructField("codigo",StringType(),True),
                          StructField("nome",StringType(),False),
                          StructField("categoria",StringType(),False),
                          StructField("faixa_preco",IntegerType(),False),
                          StructField("endereco",StringType(),False),
                          StructField("municipio",StringType(),False),
                          StructField("uf",StringType(),False),
                          StructField("codigo_bairro",StringType(),False)])

concorrentes = spark.read.option("quote","|").csv("/content/gdrive/My Drive/DesafioAlexandre/concorrentes.csv",  sep=',', inferSchema ='True', schema =concorrentesSchema, header='True' , encoding = 'UTF-8')


#Leando a fonte de dados bairros.csv
bairros = spark.read.csv("/content/gdrive/My Drive/DesafioAlexandre/bairros.csv", sep=',', inferSchema ='True', header ='True')

# Leando a fonte de dados populcao.json
# Read multiline json file
populacao = spark.read.option("multiline","true").json("/content/gdrive/My Drive/DesafioAlexandre/populacao.json")

In [ ]:
eventos.printSchema()
eventos.show(5, truncate=False)

root
 |-- codigo: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- codigo_concorrente: long (nullable = true)

+----------------------------------------------------------------+-----------------------+------------------+
|codigo                                                          |datetime               |codigo_concorrente|
+----------------------------------------------------------------+-----------------------+------------------+
|oMn07h1bJYV0Wdx+RTzsDcT8JQlT7QXc7q8A/4y+cO5gBQKOUoYGBEn1EKY7LIg4|2017-07-27 09:51:02    |650509405109544   |
|iZuQeTd9am+qfaiqnn5kkixogIbwN0nY2gtMwZqH9bFqph8qYvYchFRokjl3Pe/V|2017-06-24 14:00:26.405|650509405109544   |
|iIMvwWSnQoW0aqQlcvjc8A6LvjkRX1HLppdkdQZapPVVv7VJLDVGqzqsaBrpuu+7|2017-07-06 21:51:11.056|650509405109544   |
|jYaJZuli3fVSNDOs60if9VObBmCOoJP2x9H3IZP+6FsMRk/fZC8VHIoo683FEkKe|2017-07-02 14:27:09.316|650509405109544   |
|jYaJZuli3fVSNDOs60if9VObBmCOoJP2x9H3IZP+6FsMRk/fZC8VHIoo683FEkKe|2017-07-15 03:03:29.731|65050940

In [ ]:
#############################
concorrentes.printSchema()
concorrentes.show(20, truncate=False)

root
 |-- codigo: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- faixa_preco: integer (nullable = true)
 |-- endereco: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_bairro: string (nullable = true)

+---------------------------------------------------------------------------+-------------------------------+-------------------+-----------+-------------------------------------+---------------------+----------+-------------+
|codigo                                                                     |nome                           |categoria          |faixa_preco|endereco                             |municipio            |uf        |codigo_bairro|
+---------------------------------------------------------------------------+-------------------------------+-------------------+-----------+-------------------------------------+---------------------+----------+---------

In [ ]:
concorrentes.select("codigo_bairro").where("codigo_bairro")

+--------------------+
|       codigo_bairro|
+--------------------+
|                null|
|                  SP|
|                null|
|                  SP|
|                 Itu|
|                  SP|
|                null|
|                  SP|
|                  SP|
|                  SP|
|                  SP|
|                  SP|
|                  SP|
|      13240 Jarinu""|
|    13320010 Salto""|
|    13901-350 Amparo|
| 13920-000 Pedrei...|
|                  SP|
|           São Paulo|
|     Várzea Paulista|
+--------------------+
only showing top 20 rows



In [ ]:
bairros.printSchema()
bairros.show(10, truncate=False)

root
 |-- codigo: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- area: double (nullable = true)

+----------+------------------+-----------+---+--------+
|codigo    |nome              |municipio  |uf |area    |
+----------+------------------+-----------+---+--------+
|355620110 |Observatório      |Valinhos   |SP |68.0009 |
|3519071024|Rp 6-24           |Hortolândia|SP |0.981768|
|3536505002|Jardim De Itapoan |Paulínia   |SP |0.808537|
|3519071026|Rp 6-26           |Hortolândia|SP |2.21108 |
|3536505001|Nova Paulínia     |Paulínia   |SP |0.386199|
|3536505018|Balneário Tropical|Paulínia   |SP |1.70656 |
|3536505019|Nova Veneza       |Paulínia   |SP |1.84108 |
|35095090  |Campo Grande      |Campinas   |SP |47.0085 |
|3536505021|Jardim Itapoan    |Paulínia   |SP |3.1602  |
|3536505024|São Luiz          |Paulínia   |SP |0.12222 |
+----------+------------------+-----------+---+--------+
only showing 

In [ ]:
populacao.printSchema()
populacao.show(5, truncate=False)

root
 |-- codigo: long (nullable = true)
 |-- populacao: long (nullable = true)

+----------+---------+
|codigo    |populacao|
+----------+---------+
|355620110 |8717     |
|3519071024|5764     |
|3536505002|1195     |
|3519071026|17840    |
|3536505001|1252     |
+----------+---------+
only showing top 5 rows

